# Data Pipeline:
The Data-Pipeline notebook is used to construct a dictionary of OHLC-Dataframes (open,high,low,close).
The Data is supplied as a json file via a REST-API (Alpha Vantage).
This dictionary of dataframes will be pickled for later use.


In [1]:
finnhub_api_key = "bsuq2c748v6sd8gig46g"

In [3]:
alpha_vantage_api_key = "PO5LIXWQREXYQI7I"

In [4]:
import requests
import json
import pickle
import time 
import pandas as pd
import numpy as np
import datetime as dt



In [5]:
#symbols_df = pd.read_csv("companylist.csv")

In [6]:
#symbols_df.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,TXG,"10x Genomics, Inc.",108.7800,$10.92B,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments,https://old.nasdaq.com/symbol/txg,NaN
1,YI,"111, Inc.",6.7500,$555.93M,2018.0,Health Care,Medical/Nursing Services,https://old.nasdaq.com/symbol/yi,NaN
2,PIH,"1347 Property Insurance Holdings, Inc.",4.3285,$26.27M,2014.0,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pih,NaN
3,PIHPP,"1347 Property Insurance Holdings, Inc.",25.1000,NaN,NaN,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pihpp,NaN
4,TURN,180 Degree Capital Corp.,1.9600,$61M,NaN,Finance,Finance/Investors Services,https://old.nasdaq.com/symbol/turn,NaN


## The Symbol List:
The symbol list is used to pick our "Universe" of Stocks we wish to analyze.
These symbols will be used to get the data via the REST-API.
Here I picked:
- Apple
- Tesla
- Microsoft
- Google
- Amazon
- Wallmart 
- HomeDepot 
- J.P. Morgan Chase
- Verizon
- EXON Mobile
- American Airlines.

In [55]:
symbol_list = ["AAPL","TSLA","MSFT","GOOGL","AMZN","WMT","HD","JPM","VZ","XOM","AAL"]
#symbol_list =["ADDDF","BAYZF","DB","TSLA","VLKAF"]

It's important to note here that the data has daily-resolution and since **outputsize = compact** we only get 100 rows of data.
If we want a full history we need to change this parameter to **outputsize = full**.

In [7]:
def query_timeseries(ticker):
    alpha_vantage_api_key = "PO5LIXWQREXYQI7I"
    key = alpha_vantage_api_key
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=compact&apikey={}'.format(ticker,alpha_vantage_api_key)
    response = requests.get(url)
    res = response.json()
    return res


In [8]:
def generate_price_dfs(symbols_list):
    out = {}
    for symbol in symbols_list:       
        res = query_timeseries(symbol) #get the json output from REST-API
        time.sleep(10) #nessesary as the free version of the api has a call/minute limit 
        ts_data = res['Time Series (Daily)']
        ts_df = pd.DataFrame.from_dict(ts_data).transpose()
        ts_df = ts_df.apply(pd.to_numeric) #convert strings to numeric data types
        out.update({symbol:ts_df})
    return out

In [58]:
adj_price_dict = generate_price_dfs(symbol_list)

In [59]:
def index_mapper(index):
    trf_index = []
    for date in index:
        yyyy_mm_dd = date.split("-") #split the string in the index into year,month,date strings
        yyyy = int(yyyy_mm_dd[0]) 
        mm = int(yyyy_mm_dd[1])
        dd = int(yyyy_mm_dd[2])
        date =  dt.datetime(yyyy,mm,dd) # convert the string into a datetime object
        trf_index.append(date)
    return pd.Index(trf_index)
    
        
def change_index_to_datetime(adj_price_dict):
    for key in adj_price_dict:
        #loop over every DataFrame in the price dict and convert it's index to datetime
        df = adj_price_dict[key]
        df.set_index(index_mapper(df.index), inplace= True)
    return adj_price_dict

adj_price_dict = change_index_to_datetime(adj_price_dict)

In [60]:
def pickle_adj_price_data(price_dict):
    pickle.dump( price_dict, open( "adj_price_dict.p", "wb" ))
    #pickle the price dictionary


In [61]:
pickle_adj_price_data(adj_price_dict)